<a href="https://colab.research.google.com/github/hunterad93/projects/blob/main/carbitrage_page_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install beautifulsoup4 requests pandas

import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

url_list = [
    'https://minneapolis.craigslist.org/hnp/cto/d/saint-michael-1998-subaru-forester/7626667598.html',
    'https://minneapolis.craigslist.org/ank/cto/d/champlin-subaru-forester-as-is/7624015627.html'
]

# Create an empty DataFrame to store the extracted data
columns = ['Name', 'Description', 'Price', 'Currency', 'Latitude', 'Longitude']
df = pd.DataFrame(columns=columns)

for url in url_list:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    script_tag = soup.find('script', {'id': 'ld_posting_data'})
    json_data = json.loads(script_tag.string)

    # Extract relevant fields
    name = json_data['name']
    description = json_data['description']
    price = json_data['offers']['price']
    currency = json_data['offers']['priceCurrency']
    latitude = json_data['offers']['availableAtOrFrom']['geo']['latitude']
    longitude = json_data['offers']['availableAtOrFrom']['geo']['longitude']

    # Append the extracted data to the DataFrame
    new_row = pd.DataFrame({
        'Name': [name],
        'Description': [description],
        'Price': [price],
        'Currency': [currency],
        'Latitude': [latitude],
        'Longitude': [longitude],
    })

    df = pd.concat([df, new_row], ignore_index=True)

# Print the DataFrame
df


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,Name,Description,Price,Currency,Latitude,Longitude
0,1998 Subaru Forester,"Rust free, new tires, awd Ac works Great. 19...",3000.00,USD,45.206400,-93.659300
1,Subaru Forester 2006 - AS IS,Selling AS IS: cosmetic damage as seen in the ...,6000.00,USD,45.204562,-93.377550
